In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
import os
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

In [4]:
from scr.utils import pickle_load
from scr.preprocess.data_process import TaskProcess

In [5]:
TaskProcess().sum_file_df

,task,dataset,split,csv_path,fasta_path,pkl_path
0,annotation,scl,balanced,data/annotation/scl/balanced.csv,,
1,structure,secondary_structure,cb513,[data/structure/secondary_structure/secondary_...,,
2,proeng,gb1,two_vs_rest,data/proeng/gb1/two_vs_rest.csv,data/proeng/gb1/5LDE_1.fasta,data/proeng/gb1/two_vs_rest.pkl
3,proeng,gb1,low_vs_high,data/proeng/gb1/low_vs_high.csv,data/proeng/gb1/5LDE_1.fasta,data/proeng/gb1/low_vs_high.pkl
4,proeng,aav,one_vs_many,data/proeng/aav/one_vs_many.csv,data/proeng/aav/P03135.fasta,data/proeng/aav/one_vs_many.pkl
5,proeng,aav,two_vs_many,data/proeng/aav/two_vs_many.csv,data/proeng/aav/P03135.fasta,data/proeng/aav/two_vs_many.pkl
6,proeng,thermo,mixed,data/proeng/thermo/mixed.csv,,


In [3]:
from scr.preprocess.data_process import ProtranDataset

In [7]:
dataset_path="data/proeng/gb1/two_vs_rest.pkl"
val_dataset = ProtranDataset(dataset_path=dataset_path, subset="val")
len(val_dataset), val_dataset[0]

(43,
 ('MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVNGEWTYDDATKTFTVTELEVLFQGPLDPNSMATYEVLCEVARKLGTDDREVVLFLLNVFIPQPTLAQLIGALRALKEEGRLTFPLLAECLFRAGRRDLLRDLLHLDPRFLERHLAGTMSYFSPYQLTVLHVDGELCARDIRSLIFLSKDTIGSRSTPQTFLHWVYCMENLDLLGPTDVDALMSMLRSLSRVDLQRQVQTLMGLHLSGPSHSQHYRHTPLEHHHHHH',
  1.80292279874,
  'D40N',
  1))

In [8]:
val_dataset.x[0], val_dataset.y[0], val_dataset.mut_name[0], val_dataset.mut_numb[0]

('MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVNGEWTYDDATKTFTVTELEVLFQGPLDPNSMATYEVLCEVARKLGTDDREVVLFLLNVFIPQPTLAQLIGALRALKEEGRLTFPLLAECLFRAGRRDLLRDLLHLDPRFLERHLAGTMSYFSPYQLTVLHVDGELCARDIRSLIFLSKDTIGSRSTPQTFLHWVYCMENLDLLGPTDVDALMSMLRSLSRVDLQRQVQTLMGLHLSGPSHSQHYRHTPLEHHHHHH',
 1.80292279874,
 'D40N',
 1)

In [9]:
dataset_path = "data/annotation/scl/balanced.csv"
val_dataset = ProtranDataset(dataset_path=dataset_path, subset="val")
len(val_dataset), val_dataset[0]


(1678,
 ('MATPSAAFEALMNGVTSWDVPEDAVPCELLLIGEASFPVMVNDMGQVLIAASSYGRGRLVVVSHEDYLVEAQLTPFLLNAVGWLCSSPGAPIGVHPSLAPLAKILEGSGVDAKVEPEVKDSLGVYCIDAYNETMTEKLVKFMKCGGGLLIGGQAWDWANQGEDERVLFTFPGNLVTSVAGIYFTDNKGDTSFFKVSKKMPKIPVLVSCEDDLSDDREELLHGISELDISNSDCFPSQLLVHGALAFPLGLDSYHGCVIAAARYGRGRVVVTGHKVLFTVGKLGPFLLNAVRWLDGGRRGKVVVQTELRTLSGLLAVGGIDTSIEPNLTSDASVYCFEPVSEVGVKELQEFVAEGGGLFVGAQAWWWAFKNPGVSPLARFPGNLLLNPFGISITSQSLNPGPFRTPKAGIRTYHFRSTLAEFQVIMGRKRGNVEKGWLAKLGPDGAAFLQIPAEEIPAYMSVHRLLRKLLSRYRLPVATRENPVINDCCRGAMLSLATGLAHSGSDLSLLVPEIEDMYSSPYLRPSESPITVEVNCTNPGTRYCWMSTGLYIPGRQIIEVSLPEAAASADLKIQIGCHTDDLTRASKLFRGPLVINRCCLDKPTKSITCLWGGLLYIIVPQNSKLGSVPVTVKGAVHAPYYKLGETTLEEWKRRIQENPGPWGELATDNIILTVPTANLRTLENPEPLLRLWDEVMQAVARLGAEPFPLRLPQRIVADVQISVGWMHAGYPIMCHLESVQELINEKLIRTKGLWGPVHELGRNQQRQEWEFPPHTTEATCNLWCVYVHETVLGIPRSRANIALWPPVREKRVRIYLSKGPNVKNWNAWTALETYLQLQEAFGWEPFIRLFTEYRNQTNLPTENVDKMNLWVKMFSHQVQKNLAPFFEAWAWPIQKEVATSLAYLPEWKENIMKLYLLTQMPH',
  'Cell membrane',
  '',
  nan))

In [10]:
val_dataset.x[0], val_dataset.y[0], val_dataset.mut_name[0], val_dataset.mut_numb[0]

('MATPSAAFEALMNGVTSWDVPEDAVPCELLLIGEASFPVMVNDMGQVLIAASSYGRGRLVVVSHEDYLVEAQLTPFLLNAVGWLCSSPGAPIGVHPSLAPLAKILEGSGVDAKVEPEVKDSLGVYCIDAYNETMTEKLVKFMKCGGGLLIGGQAWDWANQGEDERVLFTFPGNLVTSVAGIYFTDNKGDTSFFKVSKKMPKIPVLVSCEDDLSDDREELLHGISELDISNSDCFPSQLLVHGALAFPLGLDSYHGCVIAAARYGRGRVVVTGHKVLFTVGKLGPFLLNAVRWLDGGRRGKVVVQTELRTLSGLLAVGGIDTSIEPNLTSDASVYCFEPVSEVGVKELQEFVAEGGGLFVGAQAWWWAFKNPGVSPLARFPGNLLLNPFGISITSQSLNPGPFRTPKAGIRTYHFRSTLAEFQVIMGRKRGNVEKGWLAKLGPDGAAFLQIPAEEIPAYMSVHRLLRKLLSRYRLPVATRENPVINDCCRGAMLSLATGLAHSGSDLSLLVPEIEDMYSSPYLRPSESPITVEVNCTNPGTRYCWMSTGLYIPGRQIIEVSLPEAAASADLKIQIGCHTDDLTRASKLFRGPLVINRCCLDKPTKSITCLWGGLLYIIVPQNSKLGSVPVTVKGAVHAPYYKLGETTLEEWKRRIQENPGPWGELATDNIILTVPTANLRTLENPEPLLRLWDEVMQAVARLGAEPFPLRLPQRIVADVQISVGWMHAGYPIMCHLESVQELINEKLIRTKGLWGPVHELGRNQQRQEWEFPPHTTEATCNLWCVYVHETVLGIPRSRANIALWPPVREKRVRIYLSKGPNVKNWNAWTALETYLQLQEAFGWEPFIRLFTEYRNQTNLPTENVDKMNLWVKMFSHQVQKNLAPFFEAWAWPIQKEVATSLAYLPEWKENIMKLYLLTQMPH',
 'Cell membrane',
 '',
 nan)

In [4]:
from torch.utils.data import DataLoader

In [5]:
from scr.encoding.encoding_classes import ESMEncoder

In [8]:
dataset_path="data/proeng/gb1/two_vs_rest.pkl"
val_dataset = ProtranDataset(dataset_path=dataset_path, subset="val")
val_loader = DataLoader(
            dataset=val_dataset,
            batch_size=32,
            shuffle=False,
            worker_init_fn=42,
        )

for val_batch in val_loader:
    mean_flat_encoder = ESMEncoder(
        encoder_name="esm1b_t33_650M_UR50S",
        embed_layer=0,
    ).encode(mut_seqs=val_batch[0], mut_names=val_batch[2],flatten_emb="mean")
    one_mean_emb = next(mean_flat_encoder)
    print(one_mean_emb.shape)

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


(32, 1280)
Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


(11, 1280)


In [ ]:
dataset_path="data/annotation/scl/balanced.csv"
val_dataset = ProtranDataset(dataset_path=dataset_path, subset="val")
val_loader = DataLoader(
            dataset=val_dataset,
            batch_size=32,
            shuffle=False,
            worker_init_fn=42,
        )
for val_batch in val_loader:
    mean_flat_encoder = ESMEncoder(
        encoder_name="esm1b_t33_650M_UR50S",
        embed_layer=0,
    ).encode(mut_seqs=val_batch[0], mut_names=val_batch[2],flatten_emb="mean")
    one_mean_emb = next(mean_flat_encoder)
    print(one_mean_emb.shape)